# BasicRNNCell
- TensorFlow 0.9 implementation based on hunkim's tutorial
- https://hunkim.github.io/ml/
- https://www.youtube.com/watch?v=A8wJYfDUYCk&feature=youtu.be
- https://gist.github.com/j-min/481749dcb853b4477c4f441bf7452195

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_rdic = ['h', 'e', 'l', 'o'] # id -> char
char_dic = {w : i for i, w in enumerate(char_rdic)} # char -> id
print (char_dic)

{'h': 0, 'e': 1, 'l': 2, 'o': 3}


In [3]:
ground_truth = [char_dic[c] for c in 'hello']
print (ground_truth)

[0, 1, 2, 2, 3]


# 데이터 준비


In [6]:
x_data = np.array([[1,0,0,0], # h
                   [0,1,0,0], # e
                   [0,0,1,0], # l
                   [0,0,1,0]], # l
                 dtype = 'f')

In [7]:
x_data = tf.one_hot(ground_truth[:-1], len(char_dic), 1.0, 0.0, -1)
print(x_data)

Tensor("one_hot:0", shape=(4, 4), dtype=float32)


tf.one_hot(indices, depth, on_value=None, off_value=None, axis=None, dtype=None, name=None)
- indices: 인덱스들의 Tensor.
- depth: One-hot 차원의 깊이(depth)를 결정하는 스칼라 값.
- on_value: indices[j] = i인 경우 채울 스칼라 값. (기본값: 1, 선택사항)
- off_value: indices[j] != i인 경우 채울 스칼라 값. (기본값: 0, 선택사항)
- axis: 채워질 축 (기본값: -1, 선택사항).
- dtype: 출력 텐서의 자료형.
- [0.0 0.0 5.0]  // one_hot(2)

indices의 인덱스에 있는 위치는 on_value, 아닌 위치는 off_value의 값을 가집니다.

In [8]:
# Configuration
rnn_size = len(char_dic) # 4
batch_size = 1
output_size = 4

# Cell 정의

In [10]:
#rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = rnn_size,
rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units = rnn_size,
                                       input_size = None, # deprecated at tensorflow 0.9
                                       #activation = tanh,
                                       )
print(rnn_cell)

#신 버젼 변경 rnn_cell->tf.nn.rnn_cell / rnn.rnn->tf.nn.rnn

# 초기 Cell 0으로 초기화

In [11]:
initial_state = rnn_cell.zero_state(batch_size, tf.float32)
print(initial_state)
# tf.zeros([batch_size, rnn_cell.state_size])도 같은 결과

Tensor("zeros:0", shape=(1, 4), dtype=float32)


# RNN Timestemp(가로축)정의

In [17]:
x_data = tf.cast(x_data, tf.int32) # 에러로 추가 
x_split = tf.split(0, len(char_dic), x_data) # 가로축으로 4개로 split
print(x_split)

[<tf.Tensor 'split_2:0' shape=() dtype=int32>, <tf.Tensor 'split_2:1' shape=() dtype=int32>, <tf.Tensor 'split_2:2' shape=() dtype=int32>, <tf.Tensor 'split_2:3' shape=() dtype=int32>]


# RNN구성

In [22]:
outputs, state = tf.contrib.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)

TypeError: 'module' object is not callable

In [ ]:
print (outputs)
print (state)

In [ ]:
logits = tf.reshape(tf.concat(1, outputs), # shape = 1 x 16
                    [-1, rnn_size])        # shape = 4 x 4
logits.get_shape()
"""
[[logit from 1st output],
[logit from 2nd output],
[logit from 3rd output],
[logit from 4th output]]
"""

In [ ]:
targets = tf.reshape(ground_truth[1:], [-1]) # a shape of [-1] flattens into 1-D
targets.get_shape()

In [ ]:
weights = tf.ones([len(char_dic) * batch_size])

In [ ]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

In [ ]:
# Launch the graph in a session
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(100):
        sess.run(train_op)
        result = sess.run(tf.argmax(logits, 1))
        print(result, [char_rdic[t] for t in result])